##  **Импорт библиотек и загрузка данных**<a  class="anchor" id="loading"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m=Mystem()
from collections import Counter

In [2]:
# загружаем данные
try:
    data=pd.read_csv(' ')
    
except:
    url='https://drive.google.com/file/d/ /view?usp=sharing'
    url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
    data = pd.read_csv(url2)

Проверим датасет на соотвествие типов данных описанию, отсутствие пропусков и дубликатов в категориях.

In [3]:
# Функция для вывода информации о структуре и типах данных датафрейма, проверки на пропуски в столбцах и дубликаты записей

def check_dataset(dataset):
    
    try:
        name =[x for x in globals() if globals()[x] is dataset][0]
        width = 10*len(dataset.columns)
        print('\033[1m','Проверка таблицы: {}'.format(name),'\033[0m')
        print('.'*width)
        print('\033[1m','\nИнформация о типах данных:', '\033[0m')
        display(dataset.info())
        print('.'*width)
        if dataset.isna().mean().sum()==0:
            print('\033[1m', '\nПропусков в столбцах \033[32;1m НЕТ', '\033[0m')
        else:
            print('\033[1m', '\nДоля пропусков в столбцах:', '\033[0m')
            display(pd.DataFrame(round((dataset.isna().mean()*100),2)).style.background_gradient())
        print('.'*width)
        if dataset.duplicated().sum()==0:
            print('\033[1m', '\nДубликатов строк \033[32;1m НЕТ', '\033[0m')
        else:
            print('\033[1m', '\nКоличество дубликатов: \033[31;1m{}\033[0m'.format(dataset.duplicated().sum()))
        print('.'*width)
        print('\033[1m', '\n10 случайных строк датафрейма', '\033[0m')
        display(dataset.sample(10))
           
    except:
        print('Ошибка')

In [4]:
check_dataset(data)

 Проверка таблицы: data 
........................................................................................................................
 
Информация о типах данных: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

........................................................................................................................
 
Доля пропусков в столбцах: 


,0
children,0
days_employed,10.1
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


........................................................................................................................
 
Количество дубликатов: 54
........................................................................................................................
 
10 случайных строк датафрейма 


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12417,0,388408.975161,64,высшее,0,вдовец / вдова,2,F,пенсионер,0,42319.912931,покупка жилой недвижимости
8120,0,NaN,36,высшее,0,женат / замужем,0,F,компаньон,0,NaN,ремонт жилью
8185,0,401021.500882,58,среднее,1,женат / замужем,0,F,пенсионер,0,84347.460897,получение образования
17418,0,398600.069018,60,Неоконченное высшее,2,женат / замужем,0,F,пенсионер,0,83625.583859,заняться высшим образованием
20789,0,-3022.880100,45,среднее,1,женат / замужем,0,M,сотрудник,0,162935.073787,на покупку своего автомобиля
15782,0,398749.827786,54,высшее,0,женат / замужем,0,M,пенсионер,0,165409.424754,свой автомобиль
20677,2,-6387.195972,34,высшее,0,женат / замужем,0,F,сотрудник,0,118203.598349,покупка своего жилья
11277,1,-415.138055,27,среднее,1,женат / замужем,0,M,сотрудник,0,77624.797300,покупка жилья для сдачи
8665,0,NaN,46,среднее,1,в разводе,3,F,госслужащий,0,NaN,получение высшего образования
8895,0,-3218.434996,31,начальное,3,гражданский брак,1,M,сотрудник,0,223933.790596,свадьба


**Вывод**

Структура данных имеет 21525 записей, содержит симметричные пропуски значений и дубликаты записей.

## **Предобработка данных**<a  class="anchor" id="preparing"></a>



---


### Обработка пропусков

Приводим значения поля об уровне образования клиентов к нижнему регистру.

In [5]:
data["education"]=data["education"].str.lower()

Исправляем артефакты в данных о количестве детей в семье (отрицательные значения и ошибки фиксации).

In [6]:
print('Данные о количестве детей в семье:','\n',data['children'].value_counts(),end='\n\n')
data['children']=data['children'].abs()
data['children']=data['children'].replace(20,2)

Данные о количестве детей в семье: 
  0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64



В 47 записях, не содержащих пропущенные значения, в поле "Количество детей в семье" содержатся артефакты: отрицательные значения. Предполагаем, что отрицательные значения - являются технической ошибкой. Производим замену на значения с противоположным знаком. 

В 76 записях, не содержащих пропущенные значения, в поле "Количество детей в семье" содержатся значения 20 детей в семье. Статистически количество семей в Российской Федерации растящих более 11 детей - 0,004%. Учитывая то факт, что в наших данных отсутствуют клиенты, имеющие более 5, но менее 20 детей, мы можем предположить, что значения в поле "Количество детей в семье" равные 20 являются ошибками фиксации и, соотвественно, подлежат замене на 2.

Избавляемся от артефактов в значениях поля "общий трудовой стаж клиента в днях" (отрицательные значения).

In [7]:
print('Количество записей, содержащих артефакты в данных о стаже клиентов: \nотрицательные значения - {:.0f}, \nзначения, превышающие возраст заемщика - {:.0f}'.format(
     data[(data['days_employed']<0)].shape[0],data[data['days_employed']>data['dob_years']*365].shape[0]),end='\n\n')
data['days_employed']=data['days_employed'].abs()

Количество записей, содержащих артефакты в данных о стаже клиентов: 
отрицательные значения - 15906, 
значения, превышающие возраст заемщика - 3445



Проверяем гипотезу о том, что значения, превышающие возраст заемщика (минус минимальный возраст трудоустройства, 14 лет) - это значения стажа в рабочих часах.

In [8]:
selection_to_check=data[data['days_employed']>(data['dob_years']-14)*365]
(((selection_to_check['dob_years']-round(selection_to_check['days_employed']/(8*365),0))>=14)).sum()

16

Гипотеза подтверждается для  0.45% исследуемых записей, гипотезу - отклоняем.

Устраняем артефакты в значениях поля "общий трудовой стаж клиента в днях" (обнуляем слишком большие значения).

In [9]:
def days_employed_real(row):
    days_employed=row['days_employed']
    if row['dob_years']<=14:
        max_exp=row['dob_years']*365
    else:
        max_exp=(row['dob_years']-14)*365
    if (days_employed>max_exp):    
        row['days_employed']=0
        return row
    return row
data=data.apply(days_employed_real,axis=1)

Проверяем данные о стаже после очистки

In [10]:
print('Данные о среднем стаже, в зависимости от типа занятости','\n',round(data.groupby('income_type')['days_employed'].mean()/365,1), end='\n\n')

Данные о среднем стаже, в зависимости от типа занятости 
 income_type
безработный        0.0
в декрете          9.0
госслужащий        9.2
компаньон          5.7
пенсионер          0.0
предприниматель    1.4
сотрудник          6.3
студент            1.6
Name: days_employed, dtype: float64



У всех записей, не содержащих пропущенные значения, в поле "Трудовой стаж в днях" содержатся артефакты: отрицательные значения и значения, превосходящие возраст клиента (минус минимальный возраст трудоустройства, 14 лет). Предполагаем, что отрицательные значения - являются технической ошибкой, а значения, превосходящие возраст клиента - ошибкой фиксации. Производим замену первых значений на значения с противоположным знаком, а последних - на 0. Проверкой будет расчет среднего стажа в зависимости от типа занятости.

Анализ пропущенных значений и выбор метода их заполнения

In [11]:
print('Количество пропущеных значений в общем количестве записей {:.0f}, а их доля - {:.1%}'
      .format(data[data['days_employed'].isna()==True].shape[0],data[data['days_employed'].isna()==True].shape[0]/data.shape[0]),end='\n\n')
print(data[(data['days_employed'].isna()==True)&(data['total_income'].isna()==True)].count(),end='\n\n')

Количество пропущеных значений в общем количестве записей 2174, а их доля - 10.1%

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64



Разобьем возраст заемщиков на 6 категорий

In [12]:
dob_level_data=[[0,'до 20 лет'],[1,'21-30 лет'],[2,'31-40 лет'],[3,'41-50 лет'],[4,'51-60 лет'],[5,'старше 60 лет']]
dob_level_columns=['dob_level_id','dob_level_name']
dob_level=pd.DataFrame(data=dob_level_data,columns=dob_level_columns)

Присвоим записям возрастные группы

In [13]:
def dob_level_set(row):
    dob_years=row['dob_years']
    dob_level_id=0
    if dob_years<20:
        dob_level_id=0
    elif dob_years<=30:
        dob_level_id=1
    elif dob_years<=40:
        dob_level_id=2
    elif dob_years<=50:
        dob_level_id=3
    elif dob_years<=60:
        dob_level_id=4
    else:
        dob_level_id=5
    row['dob_level_id']=dob_level_id
    row['dob_level_name']=dob_level.loc[dob_level_id,'dob_level_name']
    return row
data=data.apply(dob_level_set,axis=1)
data['dob_level_name'].value_counts()

31-40 лет        5743
41-50 лет        5276
51-60 лет        4542
21-30 лет        3709
старше 60 лет    2140
до 20 лет         115
Name: dob_level_name, dtype: int64

Создадим таблицу средих значений стажа в зависимости от возраста, пола и образования заемщиков

In [14]:
days_empl_pivot=round(data.pivot_table(index=['dob_level_id','gender'],columns='education',values='days_employed',
                            aggfunc=np.median),0).fillna(0)
days_empl_pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13 entries, (0, F) to (5, M)
Data columns (total 5 columns):
высшее                 13 non-null float64
начальное              13 non-null float64
неоконченное высшее    13 non-null float64
среднее                13 non-null float64
ученая степень         13 non-null float64
dtypes: float64(5)
memory usage: 734.0+ bytes


Произведем замену пропущеных значений по полю "...Стаж..." средними значениями


In [15]:
def days_empl_replase(row):
    dob_level_id=row['dob_level_id']
    gender=row['gender']
    education=row['education']
    try:
        row['days_employed']=days_empl_pivot.loc[(dob_level_id,gender),education]
        return row
    except:
        return row
data[data['days_employed'].isna()==True]=data[data['days_employed'].isna()==True].apply(days_empl_replase,axis=1)


Проверим гипотезу о равенстве доходов вне зависимости от пола

In [16]:
print('Соотношение доходов мужчин и женщин на основе медиан рядов: {:.1%}'
.format(data[data['gender']=='M']['total_income'].median()/data[data['gender']=='F']['total_income'].median()))

Соотношение доходов мужчин и женщин на основе медиан рядов: 125.0%


Проверим гипотезу о равенстве доходов вне зависимости от типа занятости

In [17]:
print("Медианные значения доходов по типам занятости:",'\n',round(data.groupby('income_type')['total_income'].median(),0))

Медианные значения доходов по типам занятости: 
 income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64


Создадим таблицу средних значений дохода в зависимости от возраста, пола, образования и типа занятости заемщиков

In [18]:
income_mean_pivot=round(data.pivot_table(index=['dob_level_id','gender','education'],
                    columns='income_type',values='total_income',aggfunc=np.median),0).fillna(0)
income_mean_pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53 entries, (0, F, высшее) to (5, M, ученая степень)
Data columns (total 8 columns):
безработный        53 non-null float64
в декрете          53 non-null float64
госслужащий        53 non-null float64
компаньон          53 non-null float64
пенсионер          53 non-null float64
предприниматель    53 non-null float64
сотрудник          53 non-null float64
студент            53 non-null float64
dtypes: float64(8)
memory usage: 3.7+ KB


Произведем замену пропущеных значений по полю "...доход" средними значениями

In [19]:
def total_income_replase(row):
    dob_level_id=row['dob_level_id']
    gender=row['gender']
    education=row['education']
    income_type=row['income_type']
    try:
        row['total_income']=income_mean_pivot.loc[(dob_level_id,gender,education),income_type]
        return row
    except:
        return row
data[data['total_income'].isna()==True]=data[data['total_income'].isna()==True].apply(total_income_replase,axis=1)

Доля записей, содержащих пропущенные значения, является высокой. Пропуски в полях "Общий трудовой стаж в днях" и "Ежемесячный доход" совпадают, а значит не являются случайными. В качестве вероятной причины появления таких пропусков может выступать: непредоставление клиентом информации или подтверждающих докуменов.
Поля являются количественными и могут быть заполнены усредненными значениями, вычисленными на основе влияющих признаков.
Влияющими на доход факторами являются: возраст, образование, должность (ключевые), тип занятости, сфера деятельности и пол (второстепенные).
В представленных данных имеются четыре из вышеуказанных признака, все являются значимыми, что позволяет достаточным основанием произвести замену пропущенных значений медианными. 

Проверим записи на аномально низкие значения возраста заемщиков

In [20]:
print("Клиенты младше 18 лет (возраст и количество)",'\n',data[(data['dob_years']<18)]['dob_years'].value_counts())

Клиенты младше 18 лет (возраст и количество) 
 0    101
Name: dob_years, dtype: int64


Создадим таблицу средих значений возраста в зависимости от типа занятости и пола заемщиков 

In [21]:
dob_mean_pivot=data.pivot_table(index=['income_type'],columns='gender',values='dob_years',aggfunc=np.mean)
dob_mean_pivot.loc[:,['F','M']]=round(dob_mean_pivot.loc[:,['F','M']],0).fillna(0)
del dob_mean_pivot["XNA"]
print(dob_mean_pivot)

gender              F     M
income_type                
безработный      45.0  31.0
в декрете        39.0   0.0
госслужащий      41.0  39.0
компаньон        40.0  39.0
пенсионер        59.0  59.0
предприниматель  27.0  58.0
сотрудник        40.0  39.0
студент           0.0  22.0


Произведем замену нулевых значений по полю "Возраст..." средними значениями

In [22]:
def dob_replase(row):
    income_type=row['income_type']
    gender=row['gender']
    try:
        row['dob_years']=dob_mean_pivot.loc[income_type,gender]
        return row
    except:
        return row
data[(data['dob_years']<18)]=data[(data['dob_years']<18)].apply(dob_replase,axis=1)

При проверке данных о возрасте клиента на значения меньшие 18, в поле "Возраст клиента в годах" выявлено 101 нулевое значение. Учитывая характер базы данных, а именно информацию о заемщиках (которые не могут быть младше 18 лет), указанные значения вероятно являются артефактами (ошибками фиксации). Однако, так как остальные поля этих записей не являются пустыми и поле "Возраст клиента в годах" не будет использовано в анализе, то значения могут быть заменены на средние в зависимости от типа занятости и пола заемщика.



---


### Замена типа данных

In [23]:
print(
    "Уникальные значения для поля 'Идентификатор уровня образования':",data['education_id'].unique(),'\n', 
    "Уникальные значения для поля 'Идентификатор семейного положения':",data['family_status_id'].unique(),'\n',
    "Уникальные значения для поля 'Имел ли задолженность по кредиту':",data['debt'].unique(),end='\n\n')
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.loc[:,['children','education_id','family_status_id','debt']]=data.loc[:,['children','education_id','family_status_id','debt']].apply(pd.to_numeric,errors='coerce',downcast='integer',axis=1)


Уникальные значения для поля 'Идентификатор уровня образования': [0 1 2 3 4] 
 Уникальные значения для поля 'Идентификатор семейного положения': [0 1 2 3 4] 
 Уникальные значения для поля 'Имел ли задолженность по кредиту': [0 1]



   При анализе структуры исходных данных обращает на себя внимание факт использования неправильного типа данных для поля "Трудовой стаж в днях": вещественное с плавающей точкой вместо целого. Также подлежит переводу в целочисленные значения столбец "Ежемесячный доход", так как дробная часть суммы дохода - не существенна для анализа.
   Значения поля "Количество детей в семье" лежат в диапазоне целых чисел от 0 до 5, полей "Идентификатор уровня образования" и "Идентификатор семейного положения" - в диапазоне целых чисел от 0 до 4, а поля "Имел ли задолженность по кредиту" - 0 или 1. Для данных полей более подходящим является тип Int8 (Малое целое от -127 до 127). Выполним указанные преобразования: 
   1. для полей "Трудовой стаж в днях" и "Ежемесячный доход" применим метод .astype с аргументом 'int' для конверсии значений в длинное целое;
   2. Для полей "Идентификатор уровня образования", "Идентификатор семейного положения" и "Имел ли задолженность по кредиту" применеи метод pd.to_numeric с аргументом downcast='integer' для конверсии значений в тип Int8.   .



---


### Обработка дубликатов

In [24]:
print("Количество дублированных записей: {:.0f}, их доля от общего числа: {:.1%}".format(data.duplicated().sum(),data.duplicated().sum()/data.shape[0]))
print("Количество уникальных значений по полю 'Ежемесячный доход': {:.0f}, и сумма: {:.0f}".
      format(len(data[data.duplicated()==True]['total_income'].unique()),
             data[data.duplicated()==True]['total_income'].unique()[0]))
print("Количество уникальных значений по полю 'Идентификатор семейного положения': {:.0f}".
      format(len(data[data.duplicated()==True]['family_status_id'].unique())))
print("Количество уникальных значений по полю 'Количество детей в семье': {:.0f}".
      format(len(data[data.duplicated()==True]['children'].unique())))

data=data.drop_duplicates().reset_index(drop=True)#удаляем дублированные записи


Количество дублированных записей: 72, их доля от общего числа: 0.3%
Количество уникальных значений по полю 'Ежемесячный доход': 22, и сумма: 126197
Количество уникальных значений по полю 'Идентификатор семейного положения': 4
Количество уникальных значений по полю 'Количество детей в семье': 3


   В данных обнаружены 71 дублированная запись. Однако, так как в структуре данных отсутствует идентификационный код клиента, а значения остальных полей могут совпасть, требуется проверка: как повлияет исключение найденных возможных дублей на качество анализа.
Некоторые из дублированных записей имеют ненулевые значения в полях 'Идентификатор семейного положения' и 'Количество детей в семье' (а нулевые значения в этих полях тоже значимы) и ненулевое значение в поле 'Ежемесячный доход'. Однако, тот факт, что доля дублей составляет всего 0.3% от общего количества записей и , вероятно, существенноне повлияет на качество анализа, позволяют произвести их исключение из базы данных методом .drop_duplicates().





---


### Лемматизация

Лемматизация по полю "Цель получения кредита" и добавление поля с лемматизированными данными

In [25]:
def purpose_lemmatize(row):
    row['purpose_lemmatized']=m.lemmatize(row['purpose'])
    return row
data=data.apply(purpose_lemmatize,axis=1)#

Создание списка уникальных лемм и очистка списка от лемм длиной менее 3х символов

In [26]:
purpose_list=[]
for lst in data['purpose_lemmatized']:
    purpose_list=purpose_list + lst
    
lemmas_counter=Counter(purpose_list)
counter_list=list(lemmas_counter)#

purpose_list_cleared=[]
for elem in counter_list:
    if len(elem)>2:
        purpose_list_cleared.append(elem)
print("Леммы в поле 'Цель получения кредита':",'\n',purpose_list_cleared,end='\n\n')

Леммы в поле 'Цель получения кредита': 
 ['покупка', 'жилье', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт']



Определение количества записей, содержащих ключевые леммы

In [27]:
main_lemmas_list=['жилье', 'автомобиль','образование','свадьба','недвижимость']
main_lemmas_total=0

for lemma in main_lemmas_list:
    main_lemmas_total=main_lemmas_total+lemmas_counter[lemma]
print("Количество записей с ключевыми леммами: {:.0f}, что составляет {:.1%}"
      .format(main_lemmas_total,main_lemmas_total/data.shape[0]))

Количество записей с ключевыми леммами: 21453, что составляет 100.0%


Создание таблицы категорий кредита и таблицы ключевых лемм в "целях получения кредита"

In [28]:
loan_category_data=[[0,'на недвижимость'],[1,'автомобильный'],[2,'образовательный'],[3,'потребительский']]
loan_category_columns=['loan_category_id','loan_category_name']
loan_category=pd.DataFrame(data=loan_category_data,columns=loan_category_columns)

main_lemmas_data=[['жилье',0],['автомобиль',1],['образование',2],['свадьба',3],['недвижимость',0]]
main_lemmas_columns=['lemma','loan_category_id']
main_lemmas=pd.DataFrame(data=main_lemmas_data,columns=main_lemmas_columns)


Назначение записям категории кредита в зависимости от наличия лемм в "целях получения кредита"

In [29]:
def loan_category_insertion(row):
    row_lemmas_list=row['purpose_lemmatized']
    for lemma in main_lemmas['lemma']:
        category_id=main_lemmas[main_lemmas['lemma']==lemma]['loan_category_id'].sum()
        if lemma in row_lemmas_list:
            row['loan_category_id']=category_id
            return row
data=data.apply(loan_category_insertion,axis=1)
data=data.merge(loan_category,on='loan_category_id',how='left')


Для категоризации данных по целям проведена лемматизация по полю "Цель получения кредита", сформирован список уникальных лемм количеством символов более 2 (для исключения предлогов, союзов, частиц и знаков переноса).
Ключевыми леммами являются следующие существительные: 'жилье', 'автомобиль','образование','свадьба','недвижимость'. Соотвественно, всем целям можно присвоить 4 категории кредитов: на недвижимость, автомобильный, образовательный, потребительский.



---


### Категоризация данных

Назначение уровней доходов в зависимости от величины ежемесячного дохода

In [30]:
def income_level_def(row):
    if row["total_income"]==0:
        row['income_level']='не указан'
        return row
    if row["total_income"]<19000:
        row['income_level']='низкий'
        return row
    if row["total_income"]<27000:
        row['income_level']='ниже среднего'
        return row
    if row["total_income"]<45000:
        row['income_level']='средний'
        return row
    if row["total_income"]<60000:
        row['income_level']='выше среднего'
        return row
    else:
        row['income_level']='высокий'
        return row

data=data.apply(income_level_def,axis=1)

Назначение категории "с детьми/бездетный" в зависимости от количества детей у заемщика

In [31]:
def children_status_def(row):
    if row["children"]>0:
        row['children_status']='есть дети'
        return row
    else:
        row['children_status']='без детей'
        return row

data=data.apply(children_status_def,axis=1)

Определение функции, рассчитывающей долю и процент кредита без просрочки по категории данных

In [32]:
def quality_categories_analysis(column_name):
    category_data=[]
    category_names=data[column_name].unique()
    for cat_name in category_names:
        category_part=round((data[data[column_name]==cat_name]['debt'].count()/data[column_name].count())*100,2)
        category_ratio=round((1-data[data[column_name]==cat_name]['debt'].sum()/data[data[column_name]==cat_name]['debt'].count())*100,2)
        category_data.append([cat_name,category_part,category_ratio])
    return category_data

Определение среднего уровня возврата кредита без просрочки по всей совокупности записей

In [33]:
weighted_avg=round(100-data['debt'].sum()/data['debt'].count()*100,2)

Рассчет зависимости возврата в срок от уровня образования заемщика

In [34]:
column_name='education'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от уровня образования заемщика (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table, end='\n\n')

Зависимость возврата в срок от уровня образования заемщика (среднее значение  91.88 %) 
              education  category_part  intime_payments_ratio
0               высшее          24.47                  94.70
1              среднее          70.72                  91.01
2  неоконченное высшее           3.47                  90.86
3            начальное           1.31                  89.01
4       ученая степень           0.03                 100.00



Рассчет зависимости возврата в срок от типа занятости заемщика

In [35]:
column_name='income_type'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от типа занятости заемщика (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table,end='\n\n')

Зависимость возврата в срок от типа занятости заемщика (среднее значение  91.88 %) 
        income_type  category_part  intime_payments_ratio
0        сотрудник          51.67                  90.43
1        пенсионер          17.84                  94.36
2        компаньон          23.67                  92.60
3      госслужащий           6.79                  94.10
4      безработный           0.01                  50.00
5  предприниматель           0.01                 100.00
6          студент           0.00                 100.00
7        в декрете           0.00                   0.00



Для проведенеия анализа зависимости между уровнем дохода и возвратом кредита в срок, необходимо ввести в данные категориальную переменную, характеризующую относительный уровень дохода. За основу возьмем данные Росстата за 2020 год о распределении населения России по величине среднедушевых денежных доходов. Исходя из статистических данных выделим 5 относительных уровней среднемесячных доходов (средняя ЗП без НДФЛ - 43025руб.):
До 19 000 рублей (низкий) — 32,8 %
19 000-27 000 рублей (ниже среднего) — 18,0 %
27 000-45 000 рублей (средний) — 24,9 %
45 000-60 000 рублей (выше среднего) — 10,2 %
свыше 60 000 рублей (высокий) — 14,1 %
Назначаем записям категорию "Уровень доходов" со значениями, в зависимости от попадания в ту или иную "вилку зарплат".
Для проведения анализа зависимости между наличием детей и возвратом кредита в срок, необходимо ввести в данные категориальную переменную, характеризующую этот статус. Назначаем записям категорию "Статус наличия детей" со значениями, в зависимости от наличия или отсутстия детей у заемщика.



## **Ответы на вопросы**<a  class="anchor" id="questions"></a>



---


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [36]:
#Зависимость возврата в срок от наличия детей в семье заемщика
column_name='children_status'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от наличия детей в семье заемщика (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table,end='\n\n')

column_name='children'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от количества детей в семье заемщика (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table,end='\n\n')

Зависимость возврата в срок от наличия детей в семье заемщика (среднее значение  91.88 %) 
   children_status  category_part  intime_payments_ratio
0       есть дети          34.32                  90.79
1       без детей          65.68                  92.46

Зависимость возврата в срок от количества детей в семье заемщика (среднее значение  91.88 %) 
    children  category_part  intime_payments_ratio
0         1          22.63                  90.83
1         0          65.68                  92.46
2         3           1.54                  91.82
3         2           9.92                  90.51
4         4           0.19                  90.24
5         5           0.04                 100.00



**Вывод**

Просматривается обратная зависимость между наличием детей в семье и соблюдением платежной дисциплины заемщиками: процент возвратов кредита в срок бездетными заемщиками выше среднего и превыщает аналогичный показатель группы-антагонита на 1,67%. Причем, с увеличением количества детей, доля заемщиков, соблюдавших платежную дисциплину, снижается (аномалия по группе "3 детей").



---


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
#Зависимость возврата в срок от семейного положения заемщика
column_name='family_status'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от семейного положения заемщика (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table, end='\n\n')


Зависимость возврата в срок от семейного положения заемщика (среднее значение  91.88 %) 
            family_status  category_part  intime_payments_ratio
0        женат / замужем          57.51                  92.45
1       гражданский брак          19.35                  90.65
2         вдовец / вдова           4.47                  93.43
3              в разводе           5.57                  92.89
4  Не женат / не замужем          13.10                  90.25



**Вывод**

Существует связь между семейным положением положением заемщиков и возвратом кредита в срок. Прямопропорциональная зависимость соблюдения платежной дисциплины для клиентов из групп, в настоящем или прошлом официально регистрировавших свой брак. Доля заемщиков, возвративших кредиты в срок, из групп "не женат / не замужем" и "гражданский брак" находится ниже среднего на 1,63-1,23% соотвественно.



---


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
#Зависимость возврата в срок от уровня дохода
column_name='income_level'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от уровня дохода (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table,end='\n\n')

Зависимость возврата в срок от уровня дохода (среднее значение  91.88 %) 
     income_level  category_part  intime_payments_ratio
0        высокий          96.22                  91.80
1  выше среднего           2.75                  94.07
2        средний           0.96                  93.66
3  ниже среднего           0.06                  91.67
4      не указан           0.02                 100.00



**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок не является очевидной, предположительно из-за принятой градации уровня дохода. В представленных данных заемщики, имеющие подтвержденный доход на уровне высокого для РФ, составляют 86,44%. Хотя и представленная стратификация уровня доходов позволяет сделать несколько выводов:
1. Заемщики, задекларировавшие доход на уровне среднего и выше, составляющие 3,71% от общего числа, отличаются значительно более высокой платежной дисциплиной по сравнению с более обеспеченными клиентами. Чей процент возврата кредитов в срок находится ниже среднего и лишь на 0,12% превышает данный показатель у наименее обеспеченной группы.
2. Отсутсвие подтвержденных доходов заемщиков не свидетельствует об их проблемах с платежной дисциплиной. Процент возвратов кредита в срок для данной группы превышает среднее значение, рассчитанное по всем заемщикам.



---


- Как разные цели кредита влияют на его возврат в срок?

In [39]:
#Зависимость возврата в срок от цели кредита
column_name='loan_category_name'
category_analysis=quality_categories_analysis(column_name)
columns=[column_name,'category_part','intime_payments_ratio']
category_analysis_table=pd.DataFrame(data=category_analysis,columns=columns)

print("Зависимость возврата в срок от цели кредита (среднее значение ",weighted_avg,"%)",'\n',category_analysis_table,end='\n\n')



Зависимость возврата в срок от цели кредита (среднее значение  91.88 %) 
   loan_category_name  category_part  intime_payments_ratio
0    на недвижимость          50.39                  92.77
1      автомобильный          20.07                  90.64
2    образовательный          18.70                  90.78
3    потребительский          10.83                  92.00



**Вывод**

Представленные данные позволяют сделать вывод о наличии связи цели кредитования и платежной дисциплины заемщика. Более взвешенная цель привлечения заемных средств (приобретение и ремонт недвижимости) находится в прямой корреляции с процентом заемщиков, возвративших кредиты в срок. В то время как цели привлечения кредита, основанные временнЫх предпочтениях потребления (авто- и потребительский кредит), и цели, основанные на ожиданиях улучшения положения заемщика (образовательный), соотносятся с процентом заемщиков, возвративших кредиты в срок, в значенн ниже среднего.  

## **Общий вывод**<a  class="anchor" id="summary"></a>



1. Существует зависимость между наличием детей в семье и соблюдением платежной дисциплины заемщиками. Наличие детей в семье заемщика снижает вероятность возврата кредита в срок. Вероятно из-за непрогнозируемых расходов, связанных с воспитанием детей.
2. Семейные люди (в формальном смысле) и люди, ранее состоявшие в зарегистрированном браке, серьезнее относятся к обязательствам по кредитам.
3. Зависимость между декларируемым заемщиком уровнем дохода, исходя из предстваленных данных, противоречит интуитивным ожиданиям: люди с высоким доходом соблюдают платежную дисциплину ниже среднего. Однако, в случае если в данных содержится не подтверженный, а декларируемый на словах уровень дохода, возможно значения ежемесячного дохода, находящиеся в группе "высоких", не связаны с реальностью.
В то же время можно считать доказанным предположение, что заемщики со средним и выше доходом отличаются более высокой платежной дисциплиной по сравнению с наименее обеспеченной группой. 
4. Кредиты не цели, имеющие быстро снижающуюся полезность для заемщика или основанные на ожиданиях изменения дохода, более рискованные в плане возврата, по сравнению с займами на недвижимость.
Исходя из вышеизложенного, можно составить портрет заемщика с потенциальной платежной дисциплиной выше среднего:
женатый, бездетный госслужащий или пенсионер со средним или выше среднего достатком, имеющий высшее образование или ученую степень, берущий кредит на ремонт или приобретение недвижимости.